In [ ]:
# gt.txtの説明
# 1:フレーム数(画像データのファイル名と対応)
# 2:物体番号
# 3:バウンディングボックスの左上のx座標
# 4:バウンディングボックスの左上のy座標
# 5:バウンディングボックスの幅
# 6:バウンディングボックスの高さ
# 7:Confidence score (0~1)
# 8:Class
# 9:Visibility 視認率。(0~1)

# 8のClassの説明
# Pedestrian 1
# Person on vehicle 2
# Car 3
# Bicycle 4
# Motorbike 5
# Non motorized vehicle 6
# Static person 7
# Distractor 8
# Occluder 9
# Occluder on the ground 10
# Occluder full 11
# Reflection 12

In [ ]:
# 連番画像の出力
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# 画像の保存先フォルダ
upload_folder = 'img\\'
# カラーマップの設定
cmap = plt.colormaps['tab20']
# gt.txtの読み込み, for文での読み込みよりも早い
df = pd.read_csv('intern_data\\MOT17-02-DPM\\gt\\gt.txt', sep = ',', encoding='cp932', header=None)

# フレームの数だけ繰り返す
for frame in range(1, 601):

    # ファイル名の作成
    filename = f'{frame:06}'
    input_filename = 'intern_data\\MOT17-02-DPM\\img1\\' + filename + '.jpg'
    upload_filename = upload_folder + filename + '.jpg'

    # input画像ファイルが存在するか確認
    if os.path.exists(input_filename):
        # 画像の読み込み
        img = cv2.imread(input_filename)
        # BGR→RGBに変換(順番入れ替え,OpenCVの仕様)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # frameのデータのみを抽出
        frame_line = df[df.iloc[:, 0] == frame]

        # データの数(行数)だけ繰り返す
        for i in range(len(frame_line)):
            # 長方形の座標を取得
            recx = frame_line.iloc[i, 2]
            recy = frame_line.iloc[i, 3]
            recw = frame_line.iloc[i, 4]
            rech = frame_line.iloc[i, 5]
            # 中心座標を求める
            reccx = int((recx * 2 + recw) / 2.0)
            reccy = int((recy * 2 + rech) / 2.0)
            # 座表の範囲を収める
            reclx = int(max(0, recx))
            recly = int(max(0, recy))
            recrx = int(min(img.shape[1], recw + recx))
            recry = int(min(img.shape[0], rech + recy))

            # 長方形の描画
            cv2.rectangle(img, (reclx, recly), (recrx, recry), (255, 0, 0), thickness=3)
            # 重心の描画
            if(0 <= reccx < img.shape[1] and 0 <= reccy < img.shape[0]):
                cv2.drawMarker(img, (reccx, reccy), (0, 255, 0), markerSize=20)
        # 画像の保存
        # BGR→RGBに変換(順番入れ替え,OpenCVの仕様)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(upload_filename, img)
    else:
        print(f"{input_filename} は存在しません。")
        break

In [15]:
import cv2
import os

image_folder = 'img'  # 画像フォルダのパス
output_video = 'movie\\output_video.mp4'  # 出力する動画ファイル名
fps = 30

# 画像ファイル名を取得し、ソートして存在確認
images = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(".jpg")]
images.sort()

# 最初の画像でフレームサイズを取得
frame = cv2.imread(images[0])
height, width, layers = frame.shape

# 動画ファイルの設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# 画像をフレームとして追加
for img_path in images:
    frame = cv2.imread(img_path)
    if frame is not None:
        video.write(frame)
    else:
        print(f"画像 {img_path} の読み込みに失敗しました。")

video.release()
print("動画の作成が完了しました。")


動画の作成が完了しました。
